In [ ]:
import os
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
import plotly.graph_objects as go


import pandas as pd
from scipy import odr
from scipy.io import readsav
from scipy.interpolate import UnivariateSpline

%run ../../Functions.ipynb

Coordinates from new TP of 304 images with STA,SDO and STB

In [ ]:
#Triangulation coordinates
tn  = pd.read_csv('scc3_304.csv')

#Central coordinates of the triangulation
Tn = tn.groupby(['T']).agg({'lonC':['median','std'], 'lat':['median','std'], 'R':['median','std']})
Tn.columns =['lonC','eLon', 'lat','eLat', 'R','eR']
Tn.reset_index(inplace=True)
Tn.lonC = np.mod(Tn.lonC,360)

xn,yn,zn, exn, eyn, ezn = Sph2CartE(Tn['R'],90-Tn['lat'],Tn['lonC'],Tn['eR'],Tn['eLat'],Tn['eLon'])  #Time: T304.T
exn = np.nan_to_num(exn)
eyn = np.nan_to_num(eyn)
ezn = np.nan_to_num(ezn)
tn304 = Tn['T']

Coordinates from old TP of 304 images with STA and SDO

In [ ]:
to = pd.read_csv('scc2_304.csv')
to = to.sort_values(by=['T'])
To = to.groupby(['T']).agg({'lonC':['median','std'], 'lat':['median','std'], 'R':['median','std']})
To.columns =['lonC','eLon', 'lat','eLat', 'R','eR']
To.reset_index(inplace=True)
To.lonC = np.mod(To.lonC,360)

xo,yo,zo, exo, eyo, ezo = Sph2CartE(To['R'],90-To['lat'],To['lonC'],To['eR'],To['eLat'],To['eLon'])  #Time: T304.T
exo = np.nan_to_num(exo)
eyo = np.nan_to_num(eyo)
ezo = np.nan_to_num(ezo)
to304 = To['T']

New GCS with non-radial tilt

In [ ]:
gcs  = pd.read_csv('gcs_param.csv')
n_gcs = len(gcs['T'])
gcc = np.ones([n_gcs,3,1])

for i in range(n_gcs):
    gcc[i] = RotCen(gcs['R'][i],gcs['k_fac'][i],gcs['lat'][i],gcs['lonC'][i],
                            gcs['Tilt'][i],gcs['Ne_tilt'][i])

gcc_R, gcc_lat, gcc_lon = Cart2Sph(gcc[:,0,0],gcc[:,1,0],gcc[:,2,0])
lst = np.array([[2.5],[gcc_lat.mean()],[gcc_lon.mean()]])
lst_c = Sph2Cart(lst[0],90-lst[1],lst[2])

df_gcs = pd.DataFrame({
    'T': gcs['T'],
    'X': gcc[:,0,0],
    'Y': gcc[:,1,0],
    'Z': gcc[:,2,0],
    'R':      gcc_R, 
    'lat':  gcc_lat, 
    'lonC': gcc_lon
})

In [ ]:
full = pd.concat([tn[['T','X','Y', 'Z']], df_gcs[['T','X','Y', 'Z']]], axis=0,ignore_index=True)

Old GCS parameters from radial model

In [ ]:
gcs_o  = pd.read_csv('gcs_old.csv',skip_blank_lines=False)
gcs_o['X'] = gcs_o['R']*np.sin((90.-gcs_o['lat'])*np.pi/180)*np.cos(gcs_o['lonC']*np.pi/180)
gcs_o['Y'] = gcs_o['R']*np.sin((90.-gcs_o['lat'])*np.pi/180)*np.sin(gcs_o['lonC']*np.pi/180)
gcs_o['Z'] = gcs_o['R']*np.cos((90.-gcs_o['lat'])*np.pi/180)

In [ ]:
full_o = pd.concat([to[['T','X','Y', 'Z']], gcs_o[['T','X','Y', 'Z']]], axis=0,ignore_index=True)
full_o = full_o.sort_values(by=['T'])
full_o =full_o[:-1]

Magnetic field lines surrounding the eruption

In [ ]:
ln  = pd.read_csv( 'openlinesn.csv',header=None,names=['R', 'lat', 'lonC'],sep='\s+',skip_blank_lines=False)
lp  = pd.read_csv( 'openlinesn0.csv',header=None,names=['R', 'lat', 'lonC'],sep='\s+',skip_blank_lines=False)
lc  = pd.read_csv('closedlines.csv',header=None,names=['R', 'lat', 'lonC'],sep='\s+',skip_blank_lines=False)

id0 = np.where((ln.R<1))[0]
ln['id'] = np.nan
ln.loc[id0, 'id'] = id0
ln['id'] = ln['id'].ffill()

id1 = np.where((lc.R<1))[0]
lc['id'] = np.nan
lc.loc[id1, 'id'] = id1
inm=lc['id'][::-1].ffill()
lc['id'] = inm[::-1]

xln,yln,zln = Sph2Cart(ln.R,ln.lat,ln.lonC)
xlc,ylc,zlc = Sph2Cart(lc.R,lc.lat,lc.lonC)
xlp,ylp,zlp = Sph2Cart(lp.R,lp.lat,lp.lonC)

Source region position from Sieyra et al. (2020)

In [ ]:
er = np.array([[1.01,90-29,110],[2.5,90-43,138]])
xe,ye,ze = Sph2Cart(er[:,0],er[:,1],er[:,2])

Spline fitting the old evolution

In [ ]:
model_xo = UnivariateSpline(full_o['T']-full_o['T'].min(),full_o['X'])
model_yo = UnivariateSpline(full_o['T']-full_o['T'].min(),full_o['Y'])
model_zo = UnivariateSpline(full_o['T']-full_o['T'].min(),full_o['Z'])

to_fit = np.linspace(0, full_o['T'].max()-full_o['T'].min(), len(full_o['T']))
xo_fit = model_xo(to_fit)
yo_fit = model_yo(to_fit)
zo_fit = model_zo(to_fit)

ro_fit,tho_fit,pho_fit = Cart2Sph(xo_fit, yo_fit, zo_fit)

Spline fitting the new evolution

In [ ]:
model_xn = UnivariateSpline(full['T']-full['T'].min(),full['X'])
model_yn = UnivariateSpline(full['T']-full['T'].min(),full['Y'])
model_zn = UnivariateSpline(full['T']-full['T'].min(),full['Z'])

tn_fit = np.linspace(0, full['T'].max()-full['T'].min(), len(full['T']))
xn_fit = model_xn(tn_fit)
yn_fit = model_yn(tn_fit)
zn_fit = model_zn(tn_fit)

rn_fit,thn_fit,phn_fit = Cart2Sph(xn_fit, yn_fit, zn_fit)

Evolution up to $2.5\,R_\odot$

In [ ]:
ss =np.where(ro_fit<=2.6)[0]
xo_end = xo_fit[ss]
yo_end = yo_fit[ss]
zo_end = zo_fit[ss]

In [ ]:
ss =np.where(rn_fit<=2.6)[0]
xn_end = xn_fit[ss]
yn_end = yn_fit[ss]
zn_end = zn_fit[ss]

3D deflected angle $\psi$

In [ ]:
th1  = np.radians(thn_fit[0]); ph1= np.radians(phn_fit[0]); th0 = np.radians(tho_fit[0]); ph0 = np.radians(pho_fit[0])
psi_n = np.arccos(np.sin(th1)*np.sin(np.radians(thn_fit))+np.cos(th1)*np.cos(np.radians(thn_fit))*np.cos(np.radians(phn_fit)-ph1))
psi_o = np.arccos(np.sin(th0)*np.sin(np.radians(tho_fit))+np.cos(th0)*np.cos(np.radians(tho_fit))*np.cos(np.radians(pho_fit)-ph0))
print(np.degrees(psi_n.max()),np.degrees(psi_o.max()))

### Figure 2

In [ ]:
fig, axs = plt.subplots(2, 2,figsize=(9,4),dpi=120,sharex=True)

axs[0, 0].set_ylabel('Latitude [$^\circ$]')
axs[0, 0].scatter(To.R,To.lat, c=To['T'],marker='p',cmap='rainbow',zorder=3,label='old TP')
axs[0, 0].scatter(gcs_o['R'],gcs_o['lat'], c=gcs_o['T'],marker='s',cmap='magma',vmin=gcs['T'].min(),vmax=gcs['T'].max(),label='old GCS')
axs[0, 0].plot(ro_fit,tho_fit,c='seagreen',zorder=2,label='old fit')
axs[0, 0].plot(rn_fit,thn_fit,'--',c='mediumvioletred',alpha=0.3,zorder=1,label='new fit')

axs[0, 0].legend()

axs[0, 1].set_ylabel('Carrington longitude [$^\circ$]')
ima=axs[0, 1].scatter(To.R,To.lonC, c=To['T'],marker='p',cmap='rainbow',zorder=3)
imb=axs[0, 1].scatter(gcs_o['R'],gcs_o['lonC'], c=gcs_o['T'],marker='s',cmap='magma',vmin=gcs['T'].min(),vmax=gcs['T'].max())
axs[0, 1].plot(ro_fit,pho_fit,c='seagreen',zorder=2)
axs[0, 1].plot(rn_fit,phn_fit,'--',c='mediumvioletred',alpha=0.3,zorder=1)


axs[1, 0].set_ylabel('Latitude [$^\circ$]')
axs[1, 0].set_xlabel('Height [$R_\odot$]')
axs[1, 0].scatter(Tn.R,Tn.lat, c=tn304,cmap='rainbow',vmin=to304.min(),vmax=to304.max(),zorder=3,label='new TP')
axs[1, 0].scatter(gcc_R,gcc_lat, c=gcs['T'],marker='D',cmap='magma',label='new GCS')
axs[1, 0].plot(rn_fit,thn_fit,c='mediumvioletred',zorder=2,label='new fit')
axs[1, 0].plot(ro_fit,tho_fit,'--',c='seagreen',alpha=0.5,zorder=1,label='old fit')
axs[1, 0].legend()

axs[1, 1].set_ylabel('Carrington longitude [$^\circ$]')
axs[1, 1].set_xlabel('Height [$R_\odot$]')
axs[1, 1].scatter(Tn.R,Tn.lonC, c=tn304,cmap='rainbow',vmin=to304.min(),vmax=to304.max(),zorder=3)
axs[1, 1].scatter(gcc_R,gcc_lon, c=gcs['T'],marker='D',cmap='magma')
axs[1, 1].plot(rn_fit,phn_fit,c='mediumvioletred',zorder=2)
axs[1, 1].plot(ro_fit,pho_fit,'--',c='seagreen',alpha=0.5,zorder=1)





plt.tight_layout(w_pad=4,h_pad=1.5 )
# rendering colorbar
plt.colorbar(ima, ax=axs.ravel().tolist())
plt.colorbar(imb, ax=axs.ravel().tolist())
plt.show()

# 3D plots

In [ ]:
#Sphere 1 solar radio coordinates
w, t = np.mgrid[0:2*np.pi:50*2j, 0:np.pi:50*1j]
Xs = np.cos(w)*np.sin(t)
Ys = np.sin(w)*np.sin(t)
Zs = np.cos(t)
# Ecuator and proyected plane of trajectory
xEc, yEc, zEc    = Sph2Cart(np.ones(100),90.*np.ones(100),np.linspace(0, 360, 100))

#Colorscale rainbow
rbow_cmap = plt.cm.rainbow(np.linspace(0.,1,len(tn304)))
rbow_plt = []
for k in range(len(tn304)):
    rbow_plt.append([k/(len(tn304)-1),'rgb'+str((int(rbow_cmap[k,0]*255), int(rbow_cmap[k,1]*255), int(rbow_cmap[k,2]*255)))])


In [ ]:
config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': 800,
    'width': 800,
    'scale':3 # Multiply title/legend/axis/canvas sizes by this factor
  }
}


data1 =[ # Sphere surface Ecuator and trajectory great circle
    go.Surface(x=Xs, y=Ys, z=Zs,surfacecolor=Xs**2 + Ys**2 + Zs**2,colorscale='gray',opacity=1., showscale=False),

    go.Scatter3d( x=xEc, y=yEc, z=zEc, mode='lines', line=dict(color='teal',width=4),showlegend=False),
    
    go.Scatter3d( x=xe, y=ye, z=ze, mode='markers+lines',showlegend=False),
    go.Scatter3d( x=xn, y=yn, z=zn,mode='markers',marker=dict(size=7,color=tn304,colorscale=rbow_plt,
                                    showscale=True,colorbar_title='Time [UT]',
                                    colorbar_tickfont_family='Times New Roman',
                                    colorbar_tickfont_size=20 ,colorbar_title_side='top',
                                    colorbar_tickmode='array'),showlegend=False),
    go.Scatter3d( x=gcc[:3,0,0], y=gcc[:3,1,0], z=gcc[:3,2,0],mode='markers',marker=dict(size=7,symbol='diamond',
                                    color=gcs['T'],colorscale='magma',
                                    showscale=False),showlegend=False),
    go.Scatter3d( x=xn_end, y=yn_end, z=zn_end, mode='lines', line=dict(color='purple',width=2),showlegend=False),
    
    # Magnetic field lines
    go.Scatter3d( x=xln, y=yln, z=zln, mode='lines', line=dict(color='mediumblue',width=1),showlegend=False),
    go.Scatter3d( x=xlc, y=ylc, z=zlc, mode='lines', line=dict(color='white',width=1),showlegend=False),
    go.Scatter3d( x=xlp, y=ylp, z=zlp, mode='lines', line=dict(color='grey',width=1),showlegend=False),
    
    ]
fig = go.Figure(data1,go.Layout(autosize=False,width=820,height=800))
fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False,bgcolor='lightgrey',aspectmode='data' )
fig.show(config=config)

## Topology guided path

In [ ]:
# Overlaying arcade of the source region at (110,29)
oa = np.where((( lc.lat  >= 89 - 29 ) & ( lc.lat  <= 91 - 29 ))   &   
             ( ( lc.lonC >= 110 - 1    ) & ( lc.lonC <=  110 + 1   )) & 
               ( lc.R    >= 1.03         ))[0]

id_oa = np.unique(lc.iloc[oa].id)
new = np.array([lc[lc.id == id].iloc[np.argmax(lc[lc.id == id].R)][:3] for id in id_oa])

d_tp = np.array([[1.02, 61, 110], new[np.argmin(new[:, 0])]])


k = 2
while k < 100:
        # Overlay check for new conditions
    oa = np.where(
            ((lc.lat  >= d_tp[-1, 1] - 1) & (lc.lat  <= d_tp[-1, 1] + 1)) &
            ((lc.lonC >= d_tp[-1, 2] - 1) & (lc.lonC <= d_tp[-1, 2] + 1)) &
            ( lc.R    >  d_tp[-1, 0]))[0]

    if len(oa) == 0:
        print('Open at: ', k)
        ob = np.where(
                ((ln.lat  >= d_tp[-1, 1] - 1) & (ln.lat  <= d_tp[-1, 1] + 1)) &
                ((ln.lonC >= d_tp[-1, 2] - 1) & (ln.lonC <= d_tp[-1, 2] + 1)) &
                ( ln.R > d_tp[-1, 0]))[0]

        id_ob = np.unique(ln.iloc[ob].id)
        print('Final crossing: ', len(id_ob))
        
        if len(id_ob) == 0: break
        ch =ln[ln.id ==id_ob[0]]
        ch2 = ch.iloc[np.where(ch.R>1.4)[0]]
        new = np.array([ch2['R'],ch2['lat'],ch2['lonC']]).T
        d_tp = np.append(d_tp, new, axis=0)

        break
    id_oa = np.unique(lc.iloc[oa].id)
    new = np.array([lc[lc.id == id].iloc[np.argmax(lc[lc.id == id].R)][:3] for id in id_oa])
    d_tp = np.append(d_tp, [new[np.argmin(new[:, 0])]], axis=0)
    k += 1


## Magnetic gradient path

In [ ]:
coordR  = readsav('coordB.sav')['rix']
coordTh = readsav('coordB.sav')['theta']
coordPh = readsav('coordB.sav')['phi']
magr    = readsav('magvB.sav')['br']
magth   = readsav('magvB.sav')['bth']
magph   = readsav('magvB.sav')['bph']

In [ ]:
iTh = np.where((np.degrees(coordTh)<=70) & (np.degrees(coordTh)>=20))[0]
iPh = np.where((np.degrees(coordPh)<=200) & (np.degrees(coordPh)>=100))[0]

th2gr,rgr,ph2gr = np.meshgrid(coordTh[iTh], coordR, coordPh[iPh])
thgr = 90-np.degrees(th2gr)
phgr = np.degrees(ph2gr)
magri = magr[:,:,iPh][:,iTh,:]
magthi= magth[:,:,iPh][:,iTh,:]
magphi= magph[:,:,iPh][:,iTh,:]
B2 = magri**2+magthi**2+magphi**2
magB = np.sqrt(B2)


In [ ]:
idr = np.argmin(np.abs(rgr[:,0,0]- 1.03))
idt = np.argmin(np.abs(thgr[0,:,0]-29))
idp = np.argmin(np.abs(phgr[0,0,:]-110))

c0 = np.array([[rgr[idr,0,0],thgr[0,idt,0],phgr[0,0,idp]]])
c_minE = np.append(c0,c0, axis=0)

for i in range (1, 92 - idr):
    A = magB[idr+i,idt-1:idt+2,idp-1:idp+2]
    pt, pp = np.unravel_index(A.argmin(), A.shape)
    idt = idt + pt - 1; idp = idp + pp - 1 
    c_minE = np.append(c_minE, [[rgr[idr+i,0,0],thgr[0,idt,0],phgr[0,0,idp]]], axis=0)
    if (idr==91):
        break

### Figure 6b

In [ ]:
plt.figure(figsize=(4,2),dpi=100)
hc = np.argmin(magB[-1], axis=1)
plt.axis('equal')
plt.axis([100,200,20,70])

plt.xlabel('Carrington Longitude [°]')
plt.ylabel('Latitude [°]')
vmin = -1.5
vmax = 1.5
cf =plt.contourf(phgr[0],90-thgr[0],magB[-1],levels=20,cmap='RdPu_r',alpha=0.5,zorder=1)
plt.plot(phn_fit, thn_fit,c='mediumvioletred',zorder=1)
plt.plot(110,29,'x',c='k')
plt.scatter(Tn.lonC,Tn.lat, c=tn304,cmap='rainbow',zorder=2)
plt.scatter(gcc_lon,gcc_lat, c=gcs['T'],marker='D',cmap='magma',zorder=2)
plt.plot(phgr[-1,0,hc],thgr[-1,:,0],':',c='w')
plt.plot(c_minE.T[2],c_minE.T[1],'--',c='seagreen',zorder=3)
plt.plot(d_tp.T[2],90-d_tp.T[1],'--',c='darkorange',zorder=3)

plt.title('$|B|$ at $r=2.5 R_\odot$' )

plt.show()


### Figure 4

In [ ]:
dm_r,dm_t,dm_p = Cart2Sph(xn_end,yn_end,zn_end)
data2 =[ 
    go.Scatter3d( x=Tn['R'], y=Tn['lonC'], z=Tn['lat'],mode='markers',marker=dict(size=8,color=tn304,colorscale=rbow_plt,
                                    showscale=True,colorbar_title='Time [UT]',
                                    #colorbar_tickfont_family='Times New Roman',
                                    colorbar_tickfont_size=20 ,
                                    colorbar_title_side='top',
                                    colorbar_tickmode='array'),showlegend=False),
    go.Scatter3d( x=gcc_R[:2], y=gcc_lon[:2], z=gcc_lat[:2],mode='markers',marker=dict(size=7,symbol='diamond',
                                    color=gcs['T'],colorscale='magma',
                                    showscale=False),showlegend=False),
    go.Scatter3d( x=dm_r, y=dm_p, z=dm_t, mode='lines', line=dict(color='mediumvioletred',width=4),showlegend=False),
    go.Scatter3d( x=[1.03], y=[110], z=[29], mode='markers', marker=dict(size=5,color='black',symbol='x'),showlegend=False),


    #    Field lines    
    go.Scatter3d(x=lp['R'],y=lp['lonC'],z=90-lp['lat'], mode='lines', line=dict(color='blue',width=1),showlegend=False),
    go.Scatter3d(x=lc['R'],y=lc['lonC'],z=90-lc['lat'], mode='lines', line=dict(color='grey',width=1),showlegend=False),
    go.Scatter3d(x=d_tp[:,0],y=d_tp[:,2],z=90-d_tp[:,1], mode='lines', line=dict(color='darkorange',dash='longdashdot',width=6),showlegend=False),
    #    Magnitude B
    go.Volume( x=rgr[:].flatten(), y=phgr[:].flatten(), z=thgr[:].flatten(), value=np.log10(magB[:].flatten()*10**4),
             isomin=2,isomax=5,opacity=0.6,opacityscale="min",surface_count=15,colorscale='greys_r', showscale=False,showlegend=False),
    go.Scatter3d( x=c_minE.T[0], y=c_minE.T[2], z=c_minE.T[1], mode='lines', line=dict(color='mediumseagreen',dash='longdashdot',width=5),showlegend=False),
    
    ]
fig = go.Figure(data2,go.Layout(autosize=False,width=1000,height=1000))
fig.update_layout(font=dict(family='Times New Roman',size=14))
        
fig.update_scenes(xaxis_visible=True, yaxis_visible=True,zaxis_visible=True,bgcolor='whitesmoke',
                 xaxis=dict(title = 'Height', backgroundcolor='whitesmoke'),
                 yaxis=dict(title = 'Carrington longitude', backgroundcolor='whitesmoke'),
                 zaxis=dict(title = 'Latitude', backgroundcolor='whitesmoke'),
                 )
#fig.write_html("fieldmag.html")
fig.show()

In [ ]:
iTh = np.where((np.degrees(coordTh)<=70) & (np.degrees(coordTh)>=20))[0]
iPh = np.where((np.degrees(coordPh)<=150) & (np.degrees(coordPh)>=90))[0]

th2gr,rgr,ph2gr = np.meshgrid(coordTh[iTh], coordR, coordPh[iPh])
thgr = 90-np.degrees(th2gr)
phgr = np.degrees(ph2gr)
magri = magr[:,:,iPh][:,iTh,:]
magthi= magth[:,:,iPh][:,iTh,:]
magphi= magph[:,:,iPh][:,iTh,:]
B2 = magri**2+magthi**2+magphi**2
magB = np.sqrt(B2)


### Figure 3

In [ ]:
data3 =[ 
    go.Scatter3d( x=Tn['R'], y=Tn['lonC'], z=Tn['lat'],mode='markers',marker=dict(size=8,color=tn304,colorscale=rbow_plt,
                                    showscale=True,colorbar_title='Time [UT]',
                                    colorbar_tickfont_size=20 ,
                                    colorbar_title_side='top',
                                    colorbar_tickmode='array'),showlegend=False),

    go.Scatter3d( x=[1.03], y=[110], z=[29], mode='markers', marker=dict(size=5,color='black',symbol='x'),showlegend=False),


    #    Field lines    
    #go.Scatter3d(x=lp['R'],y=lp['lonC'],z=90-lp['lat'], mode='lines', line=dict(color='blue',width=1),showlegend=False),
    go.Scatter3d(x=lc['R'],y=lc['lonC'],z=90-lc['lat'], mode='lines', line=dict(color='grey',width=1),showlegend=False),

    #    Magnitude B
    go.Volume( x=rgr[:35].flatten(), y=phgr[:35].flatten(), z=thgr[:35].flatten(), value=np.log10(magB[:].flatten()*10**4),
             isomin=2.5,isomax=4,opacity=0.6,opacityscale="min",surface_count=15,colorscale='RdPu_r', showscale=False,showlegend=False),
    
    ]
fig = go.Figure(data3,go.Layout(autosize=False,width=1000,height=1000))
fig.update_layout(font=dict(family='Times New Roman',size=14))
        
fig.update_scenes(xaxis_visible=True, yaxis_visible=True,zaxis_visible=True,bgcolor='whitesmoke',
                 xaxis=dict(title = 'Height', backgroundcolor='whitesmoke'),
                 yaxis=dict(title = 'Carrington longitude', backgroundcolor='whitesmoke'),
                 zaxis=dict(title = 'Latitude', backgroundcolor='whitesmoke'),
                 )
#fig.write_html("fieldmag.html")
fig.show()

### Figure 5

In [ ]:

fig, axs = plt.subplots(1, 2,figsize=(8.2,2.6),dpi=150)

axs[0].set_ylabel('Latitude [$^\circ$]')
axs[0].set_xlabel('Height [$R_\odot$]')
axs[0].set_xlim(1,2.55)

axs[0].scatter(Tn.R,Tn.lat,c=tn304,cmap='rainbow',zorder=2)
axs[0].plot(rn_fit,thn_fit,c='mediumvioletred',zorder=1)
axs[0].plot(d_tp[:,0],90- d_tp[:,1],'--',c='darkorange',zorder=1)
axs[0].plot(c_minE[:,0],c_minE[:,1],'--',c='mediumseagreen',zorder=1)
axs[0].plot(1.03,29,'x',c='black')
axs[0].scatter(gcc_R[:2],gcc_lat[:2], c=gcs['T'][:2],vmax=gcs['T'][6],marker='D',cmap='magma')



axs[1].set_ylabel('Carrington longitude [$^\circ$]')
axs[1].set_xlabel('Height [$R_\odot$]')
axs[1].set_xlim(1,2.55)
im=axs[1].scatter( Tn.R,Tn.lonC,c=tn304,cmap='rainbow',zorder=2)
axs[1].scatter(gcc_R[:2],gcc_lon[:2], c=gcs['T'][:2],vmax=gcs['T'][6],marker='D',cmap='magma')
axs[1].plot(rn_fit,phn_fit,c='mediumvioletred',zorder=1)
axs[1].plot(d_tp[:,0],d_tp[:,2],'--',c='darkorange',zorder=1)
axs[1].plot(1.03,110,'x',c='black')
axs[1].plot(c_minE[:,0],c_minE[:,2],'--',c='mediumseagreen',zorder=1)
 
fig.tight_layout()
# rendering colorbar
plt.colorbar(im, ax=axs.ravel().tolist(),orientation='vertical',label='Time [UT]')#)#plt.gca().get_children()[0], ax=axs.ravel().tolist())
plt.show()

Radial trajectory from source region

In [ ]:
xR = xe[0]*np.linspace(1,2.5,len(tn304))
yR = ye[0]*np.linspace(1,2.5,len(tn304))
zR = ze[0]*np.linspace(1,2.5,len(tn304))

Distance to eruptive path in cartesian coordinates

In [ ]:
mE_c = Sph2Cart(c_minE[:,0],90-c_minE[:,1],c_minE[:,2])
tp_c = Sph2Cart(d_tp[:,0],d_tp[:,1],d_tp[:,2])
d_t = np.ones(len(xn_end[1:]))
d_g = np.ones(len(xn_end[1:]))
d_r = np.ones(len(xn_end[1:]))

for i in range(0, len(xn_end[1:])):
    dis_t = (xn_end[1+i]-tp_c[0])**2+(yn_end[1+i]-tp_c[1])**2+(zn_end[1+i]-tp_c[2])**2
    d_t[i] = np.sqrt(np.min(dis_t))
    dis_g = (xn_end[1+i]-mE_c[0])**2+(yn_end[1+i]-mE_c[1])**2+(zn_end[1+i]-mE_c[2])**2
    d_g[i] = np.sqrt(np.min(dis_g))
    dis_r = (xn_end[1+i]-xR)**2+(yn_end[1+i]-yR)**2+(zn_end[1+i]-zR)**2
    d_r[i] = np.sqrt(np.min(dis_r))

### Figure 6a

In [ ]:
plt.rc('font', size=12) 
plt.figure(figsize=(3.5,2))
plt.plot(rn_fit[ss[10:]],np.degrees(d_r[9:]/rn_fit[ss[10:]]),'-',c='purple',lw=1, label='Radial path')
plt.plot(rn_fit[ss[10:]],np.degrees(d_g[9:]/rn_fit[ss[10:]]),'--',c='mediumseagreen',lw=2, label='Gradient path')
plt.plot(rn_fit[ss[10:]],np.degrees(d_t[9:]/rn_fit[ss[10:]]),'--',c='darkorange',lw=2, label='Topological path')


plt.hlines(0,rn_fit[ss[10]],rn_fit[ss[-1]],color='grey',ls=':',lw=2)
plt.legend()
plt.xlabel('Height [$R_\odot$]')
plt.ylabel('Distance [$^\circ$]')